In [1]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap

from titanicibmq.titanic_data import *

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install sklearn

from sklearn.decomposition import PCA

  Using cached sklearn-0.0-py2.py3-none-any.whl


In [3]:
# Load the data

datapath="data/"
data, __ = titanic(datapath=datapath)
data.sample(frac=1)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
670,1,2,1,2,3,0,3,0,4
506,1,2,1,2,2,0,3,0,4
160,0,3,0,2,2,0,1,0,6
616,0,3,0,2,1,0,1,0,6
269,1,1,1,2,3,0,2,1,2
...,...,...,...,...,...,...,...,...,...
314,0,2,0,2,2,0,1,0,4
819,0,3,0,0,2,0,4,0,0
662,0,1,0,2,2,0,1,1,2
173,0,3,0,1,1,0,1,1,3


In [4]:
X = data.drop("Survived", axis=1).values
pca = PCA(n_components=2)
Xpca = pca.fit_transform(X)

In [5]:
split_ratio = 0.2

X_train = Xpca[int(data.shape[0]*split_ratio):,:]
X_test = Xpca[:int(data.shape[0]*split_ratio),:]

y_train = data.iloc[int(data.shape[0]*split_ratio):,:]["Survived"].values
y_test = data.iloc[:int(data.shape[0]*split_ratio),:]["Survived"].values

In [6]:
# Initialize Quantum Backend
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'), shots=100)

In [7]:
# Construct VQC
optimizer = COBYLA(maxiter=100)
feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=1)
ansatz = RealAmplitudes(num_qubits=feature_map.num_qubits, reps=1)

vqc = VQC(feature_map=feature_map, 
            ansatz=ansatz, 
            loss='cross_entropy', 
            optimizer=optimizer, 
            warm_start=False, 
            quantum_instance=quantum_instance)

In [8]:
# Train the model
vqc.fit(X_train, y_train)

QiskitMachineLearningError: "Shapes don't match, predict: (2,), target: ()!"

In [4]:
# Model accuracy
acc_train = vqc.score(X_train, y_train)
acc_test = vqc.score(X_test, y_test)
print("Accuracy on training dataset: {}.".format(acc_train))
print("Accuracy on testing dataset: {}.".format(acc_test))